<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Stock_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [ ]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [ ]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-3*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [ ]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

In [ ]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [ ]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [ ]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

In [ ]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

In [ ]:
investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

print('Investment: ', investment)
print('Current: ', current)
print('Current Profit/Loss: ', curr_pnl_amount)
print('Current Profit/Loss %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate Profit/Loss: ', est_pnl_amount)
print('Estimate Profit/Loss %: ', est_pnl_percentage)

In [ ]:
tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df.sort_values(by = 'Weightage%', ascending=False)

In [ ]:
stock_ids = tmp_df['Stock'].values
for stock_id in stock_ids:
    print(stock_id)
    
    df = yfin.Ticker(stock_id).history(period='18mo', interval = "1wk")[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    print(round(df['Close'].mean()))
    print(round(df.tail(1)['Close'].values[0]))